# Model fitting and robustness

In [ ]:
%matplotlib inline

# Import packages and functions
import modelbase
import numpy as np
import pandas as pd
import os
import importlib
import sys
import re
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.patches import Rectangle
from matplotlib.colors import TwoSlopeNorm, CenteredNorm, SymLogNorm, Normalize
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy.signal import find_peaks, savgol_filter
from scipy.stats import iqr
from scipy.integrate import simpson
from scipy.optimize import minimize

from modelbase.ode import Model, Simulator, mca
from modelbase.ode import ratelaws as rl
from modelbase.ode import ratefunctions as rf

from concurrent.futures import ProcessPoolExecutor
from functools import partial
from pathlib import Path
from sympy import Matrix, lambdify, linsolve, symbols
from warnings import warn
from os import listdir
from os.path import join
from functools import reduce
from operator import mul

# Helper functions
sys.path.append("../Code")
import functions as fnc
import calculate_parameters_restruct as prm
import functions_light_absorption as lip

# Import model functions
from get_current_model import get_model
from module_update_phycobilisomes import add_OCP

In [ ]:
from functions_custom_steady_state_simulator import simulate_to_steady_state_custom, _find_steady_state, get_response_coefficients, get_response_coefficients_array, get_response_coefficients_df, calculate_ss_Q_red, get_steadystate_y0

In [ ]:
from functions_fluorescence_simulation import make_lights, make_adjusted_lights, create_protocol_NPQ, create_protocol_NPQ_short, create_protocol_noNPQ

In [ ]:
# Set the paths to save figures and objects
figure_path = Path("../Figures")
results_path = Path("../Results")

plot_format = ["svg", "png"]

# Reduce or increase the number of simulated points in appropriate analyses (1=default amount)
fraction_simulated_points = 0.3

In [ ]:
# Container for different residuals
residuals = {}

## 1) Electron pathways at different light intensities
Target: 15 electrons/PSI/s LET amounting to ~65%

In [ ]:
def get_pathways_at_lights(model, y0, lights, intens):
    sims = []
    for i, light in zip(intens, lights):
        m = model.copy()
        m.update_parameter("pfd", light)

        # Prepare for ss simulation
        exch_dict = {
            "3PGA": {"k": 10, "thresh": 1000},
            # "PG": {"k": 10, "thresh": 1},
        }
        m =fnc.add_exchange(m, exch_dict)

        s = Simulator(m)
        s.initialise(y0)
        # t,y = s.simulate(10000)
        # t,y = s.simulate_to_steady_state(tolerance=1e-2)
        s,t,y = simulate_to_steady_state_custom(s,        simulation_kwargs={
            "t_end": 1e6,
            "tolerances": [[["CBBa", "PSII", "OCP"], 1e-7], [None, 1e-6]],
            "verbose": True,
        },
        rel_norm=True, return_simulator=True, **fnc.simulator_kwargs["loose"])

        if t is None:
            raise RuntimeError(f"simulation failed for i={i:.2f}")
        else:
            sims.append(s)

    # Get the electron pathways
    pathways = pd.DataFrame({i:pd.DataFrame(fnc.get_ss_electron_pathways(s)).iloc[-1,:] for s,i in zip(sims, intens)}).T
    return pathways, sims

In [ ]:
normToPS1 = True

intens = np.linspace(100, 320, int(10 * fraction_simulated_points))
lights = [lip.light_gaussianLED(670, i) for i in intens]

# Simulate Wild Type and different mutants
# Standard model
m,y0 = get_model(check_consistency=False, verbose=False)

pathways, sims = get_pathways_at_lights(m, y0, lights, intens)
fnc.save_Simulator_dated(sims, f"resid_epaths_sims", results_path)
fnc.save_obj_dated(pathways, "resid_epaths_paths", results_path)

# Get the mean LET fraction
let_frac = (pathways.T / pathways.sum(axis=1)).T["linear"]

# Residuals with target value 65%
residuals["LET_fraction"] = np.linalg.norm(let_frac - 0.65, ord=2)

# Get the Let flux per PSI
norm = m.get_parameter("PSItot") *3# Normalise to PS1 monomers
let_flux = pathways["linear"].iloc[-1] / norm

# Residuals with target value 65%
residuals["LET_flux"] = np.abs(let_flux - 15)

In [ ]:
residuals

In [ ]:
let_frac

## 2) Schuurmans Oxygen and CO2 fluxes (Figure 6)

In [ ]:
def get_O2andCO2rates(s):
    #  Get O2 and CO2 rates
    rates = {}
    res = {}

    rates["O2"] = fnc.get_stoichiometric_fluxes(s, "O2")
    rates["O2"].pop("vO2out")

    rates["CO2"] = fnc.get_stoichiometric_fluxes(s, "CO2")
    rates["CO2"].pop("vCCM")

    for cmp in ["O2", "CO2"]:
        prod = pd.DataFrame(rates[cmp].copy())
        prod[prod<0] = 0
        res[f"{cmp}_production"] = prod.sum(axis=1, skipna=False)

        cons = pd.DataFrame(rates[cmp].copy())
        cons[prod>0] = 0
        res[f"{cmp}_consumption"] = cons.sum(axis=1, skipna=False)

        res[f"{cmp}_net"] = pd.DataFrame(rates[cmp]).sum(axis=1, skipna=False)
    
    return pd.DataFrame(res)

In [ ]:
# Oxygen change rates from Schuurmans2014
O2_ref = pd.read_csv("../Code/data/O2rates_Schuurmans2014.csv", skiprows=1, index_col=0)

In [ ]:
# Reference values for plot
# Oxygen change rates from Schuurmans2014
O2_ref = pd.read_csv("../Code/data/O2rates_Schuurmans2014.csv", skiprows=1, index_col=0)

# Define the lights
intens = np.unique(np.sort(np.concatenate((np.linspace(0, 320, 30), O2_ref.index.to_numpy()))))
lights = [lip.light_gaussianLED(625, i) for i in intens]

# Standard model
m0,y0 = get_model(check_consistency=False, verbose=False)

# Standard model
mlowCO2,y0 = get_model(check_consistency=False, verbose=False)
mlowCO2.update_parameter("CO2ext_pp", 0.0004)

# Adjust the lights to in-culture conditions (2 mg(Chl) l^-1 according to Schuurmans)
MChl = 893.509 # [g mol^-1]
absorption_coef = lip.get_pigment_absorption(m0.parameters["pigment_content"]).sum(axis=1) * MChl
lights = lip.get_mean_sample_light(
    lights, # [µmol(Photons) m^-2 s^-1]
    depth = 0.01,  # [m]
    absorption_coef=absorption_coef,
    chlorophyll_sample=(
        2 # [mg(Chl) l^-1] (Schuurmans2014)
        / MChl # [mol g^-1]
        * 1e3
    ) # [mmol(Chl) m^-3]
)

models = {"m0":m0, "mlowCO2":mlowCO2}
pathways = {}
sims = {}
gasrates = {}

if settings["O2_CO2_fluxes"]["recalculate"]:
    for key,model in models.items():
        print(key)
        _pathways, _sims = get_pathways_at_lights(model, y0, lights, intens)
        pathways[key] = _pathways
        sims[key] = _sims

        # Get the O2 and CO2 rates for Wild Type
        gasrates[key] = pd.concat([get_O2andCO2rates(s).iloc[-1,:] for s in _sims], axis=1)
        gasrates[key].columns = [f"{i:.2f}" for i in intens]
    fnc.save_obj_dated(gasrates, "gasrates_validation", results_path)

## 3) Benschop O2 data

In [ ]:
# Adapt a model so that no compounds accumulate or drain
# Define a dictionary with all compounds that should be held constant
exch_dict = {
    "3PGA": {"k": 10, "thresh": 1000},
    }

# Define the MCA model by adding a flux keeping 3PGA constant
m_MCA,y0 = get_model(check_consistency=False, verbose=False)
m_MCA = fnc.add_exchange(m_MCA, exch_dict)

# Define tested CO2 concentrations and CCM factors
CO2_concs = np.logspace(np.log10(0.00004), np.log10(0.05), 10)  
CCM_factors = np.logspace(2,3,10)

O2s = pd.DataFrame(np.nan, index=CO2_concs, columns=CCM_factors)

for CO2_conc in CO2_concs:
    for CCM_factor in CCM_factors:
        if settings["CO2_fluxes_spectral"]["recalculate"]:
            m_MCA.update_parameters({
                "CO2ext_pp": CO2_conc,
                "fCin": CCM_factor
            })

            _O2s = get_ssflux(
                m_MCA,
                y0,
                lip.light_spectra,
                "vO2out",
                ("cool_white_led", 800),
                tolerance=1e-4,
                rel_norm=False
            )

            O2s.loc[CO2_conc, CCM_factor] = _O2s

fnc.save_obj_dated(O2s, "rev_ss_O2s_CO2vary", results_path)

In [ ]:
# Fit to the Benschop2003 CO2 fixation data
# Benschop2003_high = pd.read_csv(Path("data/CO2fixation_Benschop2003_highCO2.csv"), index_col=0)
Benschop2003_low = pd.read_csv(Path("data/CO2fixation_Benschop2003_lowCO2.csv"), index_col=0)

In [ ]:
# Calculate the theoretical dissolved CO2 medium concentrations at the external partial pressures
# Assuming T=30°C, S=35 (Salinity of seawater)
T= 303.150 # [K]
S = 35 # [unitless] Salinity
CO2_dis = CO2sol(T,S,CO2_concs) # [mol l^-1]
CO2_dis = CO2_dis * 1e6 # [µmol l^-1]

CO2_conv_fac = CO2sol(T,S,1) * 1e6

O2s_conv = O2s * unit_conv(['mmol mol(Chl)-1 -> mmol g(Chl)-1', 's-1 -> h-1'])

fig, ax = plt.subplots()
for i, CCM_factor in enumerate(CCM_factors):
    ax.plot(CO2_dis, O2s_conv.loc[:,CCM_factor].to_numpy(), label=f"{CCM_factor:.0f}")
    ax.set_xscale("log")


# Add values from Benschop2003
ax2 = ax.twinx()
# ax2.plot(Benschop2003_high.index.to_numpy(), Benschop2003_high.to_numpy(), "ko")
ax2.plot(Benschop2003_low.index.to_numpy(), Benschop2003_low.to_numpy(), "ko", label = "Measured CO$_2$ fixation")
ax2.legend(loc='upper left')
ax.set_ylim(0)

ax.axvline(CO2sol(T,S,1) * 1e6 * 0.0004, ls="--", c="k")
ax.text(11, 5, "400 ppm", ha="left")

ax.legend(loc='lower right', title="Parameter fCin")

ax.set_xlabel("External dissolved CO$_2$ concentration [µM]")
ax.set_ylabel("Simulated CO$_2$ fixation rate [mmol g(Chl)$^{-1}$ h$^{-1}$]")
ax2.set_ylabel("Measured CO$_2$ fixation rate [mmol g(Chl)$^{-1}$ h$^{-1}$]")

fnc.savefig_dated(fig, f"CO2variation_Benschop", type=plot_format, path=figure_path, bbox_inches="tight")

## 4) Original 435 nm PAM-SP experiment

### Unfinished

In [ ]:
def get_PS_concentration(f_PS1_PS2, fraction_is_monomers=False, output_is_monomers=False):
    if not fraction_is_monomers:
        n_chl_PSII = ( # [chl PSII^-1] number of chlorophyll molecules per PSII dimer complex(Fuente2021)
            35 * # [chl PSII^-1] number of chlorophyll molecules per PSII complex(Fuente2021)     
            2) # [unitless] two PSII monomers per dimer
        n_chl_PSI = ( # [chl PSI^-1] number of chlorophyll molecules per PSI trimer complex (Fuente2021)     
            96 * # [chl PSI^-1] number of chlorophyll molecules per PSI complex (Fuente2021)     
            3) # [unitless] three PSI monomers per trimer
    else:
        n_chl_PSII = 35 # [chl PSII^-1] number of chlorophyll molecules per PSII complex(Fuente2021)     
        n_chl_PSI = 96 # [chl PSI^-1] number of chlorophyll molecules per PSI complex (Fuente2021)     

    f_PSI_PStot = 1 / (1 + 1/f_PS1_PS2) # [PSII PStot^-1] fraction of PSIIs compared to the whole number of photosystems
    f_PSII_PStot = 1 - f_PSI_PStot # [PSI PStot^-1] fraction of PSIs compared to the whole number of photosystems

    c_PSIItot = ( # [mmol mol(Chl)^-1] total concentration of photosystem 2 complexes (ADAPT DATA SET)
        (f_PSII_PStot * n_chl_PSII) / (f_PSII_PStot * n_chl_PSII + f_PSI_PStot * n_chl_PSI) *  # [unitless] fraction of chlorophylls bound in PSII complexes
        1/n_chl_PSII * 1e3) # [PSII chl^-1] * [mmol mol^-1]
    c_PSItot = ( # [mmol mol(Chl)^-1] total concentration of photosystem 1 complexes (ADAPT DATA SET)
        (f_PSI_PStot * n_chl_PSI) / (f_PSII_PStot * n_chl_PSII + f_PSI_PStot * n_chl_PSI) *  # [unitless] fraction of chlorophylls bound in PSII complexes
        1/n_chl_PSI * 1e3) # [PSI chl^-1] * [mmol mol^-1]

    if not output_is_monomers and fraction_is_monomers:
        c_PSIItot = c_PSIItot/2
        c_PSItot = c_PSItot/3

    return {"PSIItot":c_PSIItot, "PSItot":c_PSItot}

### Get the PAM and strain data

In [ ]:
def get_pbs_attachment(df, light, rel_fluo):
    res = df.loc[:,str(light)] / pd.Series(rel_fluo)
    return (res / res.sum()).to_dict()

def get_strain_parameters(df_pigments, df_PBS, df_PS1_PS2, growthlight, pbs_relfluo):
    # Get the PBS attachment and pigments
    pbs = get_pbs_attachment(df_PBS, growthlight, pbs_relfluo)

    pigment = df_pigments.loc[:,str(growthlight)]

    # Get the photosysystems concentration form the ratio and chla content
    ps_conc = get_PS_concentration(float(df_PS1_PS2.loc[:,str(growthlight)]), fraction_is_monomers=True, output_is_monomers=False)

    # Return a combines parameter dict
    res = {"pigment_content": pigment}
    res.update(pbs)
    res.update(ps_conc)
    return res

def _plot_model_and_data(s, data, sim_offset=None, data_offset=None, ax=None):
    # Initialise a plot
    if ax is None:
        fig, ax = plt.subplots(**fnc.plt_defaults["figure"])
    else:
        fig = ax.figure

    # Align data and model using an offset
    if sim_offset is None:
        time = s.get_time()
    elif s.get_time()[0] != (-sim_offset):
        time = np.array(s.get_time())- (s.get_time()[0] + sim_offset)
    else:
        time = s.get_time()

    # Plot the data
    dat = data/data.max().max()
    dat_time = dat.index

    if data_offset is not None:
        dat_time -= data_offset
    dat_line = ax.plot(dat_time, dat.values, c="firebrick", label="Measurement")
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Fluorescence [rel.]")
    ax.set_ylim(ymin=0)

    # Plot the simulation on top
    fluo = s.get_full_results_dict()["Fluo"]
    fluo_norm = fluo / fluo.max()
    sim_line = ax.plot(time, fluo_norm, c="black", label="Simulation")

    if len(dat_line)>1:
        handles = sim_line + dat_line[0]
    else:
        handles = sim_line + dat_line

    ax.legend(
        handles = handles,
        loc="center right",
        bbox_to_anchor = (1,0.85)
    )

    return fig, ax

def plot_model_and_data(s, data, pulse_pfd, ax=None, sim_offset=None, data_offset=None, **lightbar_kwargs):
    figs={}

    # Data plot
    col_select_kw = "^F"
    col_select = [i for i,x in enumerate(data.columns) if re.search(col_select_kw, x)]
    col_selectnam = data.columns[col_select]
    plot_data = data.loc[10:data.index[-1]-10, col_selectnam[:-1]]

    s_trim = fnc.remove_beginning_from_simulation(s, 300)

    fig,ax = _plot_model_and_data(s_trim, plot_data, sim_offset=sim_offset, data_offset=data_offset, ax=ax)
    ax = fnc.add_lightbar(s_trim, ax, pulse_pfd * 0.95, color="mono", remove_pulses=True, scale="log", size=0.06, time_offset=0, **lightbar_kwargs)

    return fig, ax

def annotate_electron_pathways(ax, epath):
    ypos = epath.cumsum(axis=1) - (epath/2)

    for y,s in zip(ypos.to_numpy().flatten(), epath.to_numpy().flatten()):
        ax.text(x=0, y=y, s=np.round(s,3), ha='center', va='center')
    return ax

def ss_analysis(m, light_int=100, light_wl=659):
    sslight = lip.light_gaussianLED(light_wl, light_int)
    mss = m.copy()
    mss.update_parameter("pfd", sslight)

    # Initialise the model
    sss=Simulator(mss)
    sss.initialise(y0)
    t,y = sss.simulate(10000)

    # plot_overview_isoATP(sss)
    epath = fnc.get_ss_electron_pathways(sss)
    epath = epath/float(epath.sum(axis=1, skipna=False))
    ax = epath.plot(kind="bar", stacked=True)

    # Annotate the bars
    ax = annotate_electron_pathways(ax,epath)
    return sss, [ax], epath

In [ ]:
# Get the fluorescence data paths
DIRPATH = Path("../Code/data")
PAMPATH = DIRPATH / "PAM_Slow_kinetics_files_Zavrel2021"
file_names = listdir(PAMPATH)
file_paths = np.array([join(PAMPATH, file_name) for file_name in file_names])

In [ ]:
# Extract the measurement type from the file name, use to group replicates
meas_types = np.array([re.sub(r"([0-9]+)_[0-9]+\.[csvCSV]+", "\\1", x) for x in file_names])
meas_types_set = list(set(meas_types))
meas_types_set.sort()

In [ ]:
# Load and preprocess all data
pamdata = {}

for meas_select in  meas_types_set:
    file_paths_select = file_paths[meas_types == meas_select]

    # Create a container for the combined data
    meas_data = pd.DataFrame([])

    # Iterate through the files
    for i,file_path in enumerate(file_paths_select):
        # Adapt the file reading parameters depending on the file structure
        if meas_select.startswith("PSI+PSII kinetics"):
            skiprows = 1
        else:
            skiprows = 0
        
        # Read the file and remove empty columns
        file_data = pd.read_csv(file_path, sep=";",skiprows=skiprows, index_col=0)
        file_data=file_data.loc[:,np.invert(np.all(np.isnan(file_data), axis=0))]

        if i == 0:
            # Save the common column names
            col_names = file_data.columns

        # Combine replicates
        meas_data = meas_data.join(file_data, rsuffix=f"_{i}", how="outer")

    # Exclude all data points that are nor present in all datasets
    excl_data = meas_data.iloc[list(np.invert(np.isnan(meas_data).any(axis=1))),:]
    excl_data.shape
        
    # Normalise the Data to its highest point
    norm_data = excl_data.copy()
    norm_data = norm_data / norm_data.max()

    # Set the normalised data as analysis data
    data = norm_data

    # # Calculate the mean over selected columns
    # for col_name in col_names:
    #     col_select = [bool(re.search(col_name, x)) for x in data.columns]
    #     col_mean = data.iloc[:,col_select].mean(axis=1, skipna=False).rename(f"{col_name}_mean")

    #     # Add to the dataframe
    #     data = data.join(col_mean)
    
    # Store the data
    pamdata[meas_select] = data

# Extract the growth lights
growthlights = set([re.sub("^.*([0-9]{3})$","\\1",meas_select) for meas_select in meas_types_set])

In [ ]:

# Get the strain-specific parameters
dat = pd.read_csv(DIRPATH / "Strainparameters_Zavrel2021.csv", skiprows=1, index_col=0).iloc[:,2:]

# Get the concentration data
# dat_conc = dat.loc[dat.loc[:,"Unit"].str.match("fg cell-1"), :]
dat_conc = dat.loc[["Chlorophyll a", "β-carotene ", "Allophycocyanin", "Phycocyanin"], :]
dat_conc.index = ['chla', 'beta_carotene', 'allophycocyanin', 'phycocyanin']

# Convert to [mg mg(Chla)^-1]
dat_conc = dat_conc.div(dat_conc.iloc[0,:]) # convert to [mg mg(Chla)^-1]
dat_conc.loc[:,"Unit"] = "mg mg(Chla)^-1"

# Get the photosystems data
dat_PS = dat.loc[["PSI / PSII"],:]

# Get the phycobilisome data
dat_PBS = dat.loc[["PBS attached to PSII", "PBS attached to PSI", "PBS uncoupled - disattached from both PSI and PSII"], :]
dat_PBS.index = ["PBS_PS2", "PBS_PS1", "PBS_free"]


# Calculate the strain parameters
strain_params = {}
strain_export = {}

for growthlight in dat.columns:
    # Get the strain specific parameters
    strain_param = get_strain_parameters(
        df_pigments=dat_conc,
        df_PBS=dat_PBS,
        df_PS1_PS2=dat_PS,
        growthlight=growthlight,
        pbs_relfluo={"PBS_PS2":1, "PBS_PS1":1, "PBS_free":10},
    )

    MChl = 893.509 # [g mol^-1]
    absorption_coef = lip.get_pigment_absorption(strain_param["pigment_content"]).sum(axis=1) * MChl # [m^2 mmol(Chl)^-1]

    # Get the PAM sample Chl concentration
    # Estimated from a later measurement where OD600 = 0.8, the PAM samples had 0.6
    OD800_Chlconc = pd.read_excel(DIRPATH/"Chlorophyll, total car.xlsx", skiprows=1).iloc[1:,].loc[:,["Sample", "Chlorophyll a"]]
    OD800_Chlconc.loc[:,"Sample"] = OD800_Chlconc.loc[:,"Sample"].str.removesuffix(" nm")
    OD800_Chlconc = OD800_Chlconc.groupby("Sample").mean()
    OD800_Chlconc.head() # [mol (10 ml)^-1]

    indexmapping = {idx1:idx2 for idx1,idx2 in zip(dat.columns, OD800_Chlconc.index)}

    # Convert the chlorophyll concentration units
    cuvette_Chlconc = ( # [mmol(Chl) m^-3]
        float(OD800_Chlconc.loc[indexmapping[str(growthlight)]]) # [mol (10 ml)^-1]
        / 10 # [ml]
        * 1e3 # [mol mmol^-1]
        * 1e6 # [ml m^-3]
        * 0.3 / 0.8 # [rel] OD of the PAM sample
    )

    # Save the data
    strain_params[growthlight] = {
        "params": strain_param,
        "cuvette_Chlconc": cuvette_Chlconc
    }

### Model the data

In [ ]:

lights = make_lights(blue_wl=480)
lights_adj = make_adjusted_lights(absorption_coef, cuvette_Chlconc)
dark_adj, low_blue_adj, high_blue_adj, orange_adj, pulse_orange_adj, pulse_blue_adj = lights_adj

# Get the pfd of the pulses
pulse_pfd = np.min([simpson(x) for x in [pulse_orange_adj, pulse_blue_adj]])

In [ ]:
def complex_absorption(pfd, ps_ratio, pigment_content):
    absorption = lip.get_pigment_absorption(pigment_content)
    association = lip.get_pigment_association(
        ps_ratio,
        beta_carotene_method="stoichiometric",
        pigments=pigment_content,
        verbose=False,
    )

    M_chl = 893.509  # [g mol^-1] molar mass of chlorophyll a (C55H72MgN4O5)

    return (
        lip.get_complex_absorption(pfd, absorption, association) * M_chl
    )  # [µmol(Photons) mmol(Chl)^-1 s^-1]

### Simulate all of the datasets (435 & 633nm light in two light protocols)

In [ ]:
sims = {}

# Select the measurement to simulate
for meas_select in meas_types_set:
    growthlight = int(meas_select[-3:])

    print("Measurement: ", meas_select, "\nSimulated light: ", growthlight,"\n")

    strain_param = strain_params[str(growthlight)]["params"]
    cuvette_Chlconc = strain_params[str(growthlight)]["cuvette_Chlconc"]

    # Get the latest model version
    m4,y0 = get_model(check_consistency=False, reduced_size=False)

    # ADAPTION TO THE STRAIN
    y0.update({"PSII":strain_param["PSIItot"]})
    m4.update_parameters(
        strain_param
    )

    # Change the CO2 concentration to 400ppm as experiments were conducted in air
    m4.update_parameter("CO2ext_pp", 0.0004)

    # Initialise the model
    s4=Simulator(m4)
    s4.initialise(y0)

    # Simulate the appropriate protocol
    pulse_pfdm4 = 2600 * 2
    lights_lowpulse = make_adjusted_lights(
        absorption_coef=absorption_coef, 
        chlorophyll_sample=cuvette_Chlconc, 
        lights=make_lights(pulseInt=pulse_pfdm4, blue_wl=480)
    )
    if meas_select.startswith("PSII kinetics, NPQ at state 2"):
        protocol4 = create_protocol_noNPQ(*lights_lowpulse)
    else:
        protocol4 = create_protocol_NPQ(*lights_lowpulse)

    if settings["PAM_blue_light"]["recalculate"]:
        s4 = fnc.simulate_protocol(s4, protocol4, retry_unsuccessful=True)
        fnc.save_Simulator_dated(s4, f"Zavrel2021sim{meas_select[-5:]}", path=results_path, save_fluxes=True, save_full_results=True)
    else:
        print("loading simulation")
        s4 = fnc.load_Simulator_dated(name = f"Zavrel2021sim{meas_select[-5:]}", path=results_path, date=None)
    
    sims[meas_select] = s4

## 5) Validation PAM-SP experiment

### Unfinished

In [ ]:
df = pd.read_csv(Path("../Code/data/PAM_validation_Zavrel2024.csv"), index_col=[0], header=[0,1,2])
idx = pd.IndexSlice

In [ ]:
# Protocol of the validation experiment
dark, low_blue, high_blue, orange, pulse_orange, pulse_blue = make_lights(
    blueInt=80,
    orangeInt=50,
    highblueInt=1800,
    pulseInt=15000,
    orange_wl=625,
    blue_wl=440,
)
pulse_white = lip.light_spectra("cool_white_led", 15000)


# Dark acclimation
protocol = fnc.create_protocol_const(
    light=dark,
    time=300
)

# Dark phase
protocol = fnc.create_protocol_PAM(
    init=protocol,
    actinic=(dark, 30-0.6),  # Actinic light intensity and duration
    saturating=(
        pulse_white,
        0.6,
    ),  # Saturating pulse light intensity and duration
    cycles=4,
    final_actinic_time=5
)

# Blue phase
protocol = fnc.create_protocol_PAM(
    init=protocol,
    actinic=(low_blue, 30-0.6),  # Actinic light intensity and duration
    saturating=(
        pulse_white,
        0.6,
    ),  # Saturating pulse light intensity and duration
    cycles=6,
    first_actinic_time=25,
    final_actinic_time=5
)

# Orange phase
protocol = fnc.create_protocol_PAM(
    init=protocol,
    actinic=(orange, 30-0.6),  # Actinic light intensity and duration
    saturating=(
        pulse_white,
        0.6,
    ),  # Saturating pulse light intensity and duration
    cycles=10,
    first_actinic_time=25,
    final_actinic_time=5,
)

# Blue phase
protocol = fnc.create_protocol_PAM(
    init=protocol,
    actinic=(low_blue, 30-0.6),  # Actinic light intensity and duration
    saturating=(
        pulse_white,
        0.6,
    ),  # Saturating pulse light intensity and duration
    cycles=6,
    first_actinic_time=25,
    final_actinic_time=5
)

# High blue phase
protocol = fnc.create_protocol_PAM(
    init=protocol,
    actinic=(high_blue, 30-0.6),  # Actinic light intensity and duration
    saturating=(
        pulse_white,
        0.6,
    ),  # Saturating pulse light intensity and duration
    cycles=6,
    first_actinic_time=25,
    final_actinic_time=5
)

# Blue phase
protocol = fnc.create_protocol_PAM(
    init=protocol,
    actinic=(low_blue, 30-0.6),  # Actinic light intensity and duration
    saturating=(
        pulse_white,
        0.6,
    ),  # Saturating pulse light intensity and duration
    cycles=6,
    first_actinic_time=25,
    final_actinic_time=5
)

# Simulate the validation experiment
m,y0 = get_model()
s = Simulator(m)
s.initialise(y0)

# The culture is grown under 1% CO2
s.update_parameter("CO2ext_pp", 0.01)

s = fnc.simulate_protocol(s, protocol, retry_unsuccessful=True)

In [ ]:
idx = pd.IndexSlice
_s = fnc.remove_beginning_from_simulation(s, 299)

fig,ax= plt.subplots(figsize=(15,7))
dat = df.loc[:1126,idx["25", "white",:]]
dat_line = ax.plot(dat.index.to_numpy() + 10, dat.values, c="firebrick", label="Measurement", alpha=0.3)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Fluorescence [rel.]")
ax.set_ylim(ymin=0)

# Plot the simulation on top
fluo = _s.get_full_results_dict()["Fluo"]
fluo_norm = fluo / fluo.max()
sim_line = ax.plot(_s.get_time(), fluo_norm, c="black", label="Simulation")

handles = sim_line + [dat_line[0]]

ax.legend(
    handles = handles,
    loc="center right",
    bbox_to_anchor = (1,0.85)
)

ax = fnc.add_lightbar(_s, ax, pulse_pfd * 0.95, color="mono", remove_pulses=True, scale="log", size=0.06, time_offset=0)
ax.set_title("")
fnc.savefig_dated(fig, f"pamspplot_validation_singleplot", type=plot_format, path=figure_path, bbox_inches="tight")

In [ ]:
idx = pd.IndexSlice
_s = fnc.remove_beginning_from_simulation(s, 299)

fig,axes= plt.subplots(1,2, figsize=(15,5))
dat = df.loc[:1126,idx["25", "white",:]]

colors = mpl.colormaps["Reds"](np.linspace(0.5,1,len(dat.columns)))
for i in range(len(dat.columns)):   
    dat_line = axes[0].plot(dat.index.to_numpy() + 10, dat.iloc[:,i], c=colors[i], label="Measurement")
axes[0].set_ylabel("Fluorescence [rel.]")
axes[0].set_title("Measurement (four replicates)")


# Plot the simulation on the side
fluo = _s.get_full_results_dict()["Fluo"]
fluo_norm = fluo / fluo.max()
sim_line = axes[1].plot(_s.get_time(), fluo_norm, c="black", label="Simulation")
axes[1].set_title("Simulation")

for ax in axes:
    ax.set_xlabel("Time [s]")
    ax.set_ylim(ymin=0)
    ax = fnc.add_lightbar(_s, ax, pulse_pfd * 0.95, color="mono", remove_pulses=True, scale="log", size=0.06, time_offset=0, annotation_twoline=False)
    
    
    # Add phaseboxes
    ax = add_phasebox(ax, 0, 125, "Dark", 0.05, 1.02, adj_plotheight=True)
    ax = add_phasebox(ax, 125, 305, "State 1", 0.05, 1.02)
    ax = add_phasebox(ax, 305, 605, "State 2", 0.05, 1.02)
    ax = add_phasebox(ax, 605, 785, "State 1", 0.05, 1.02)
    ax = add_phasebox(ax, 785, 965, "NPQ activation", 0.05, 1.02)
    ax = add_phasebox(ax, 965, 1145, "NPQ relaxation", 0.05, 1.02)

# Share the y axis
axes[0].sharey(axes[1])
# axes[1].set_yticklabels([])

fig.tight_layout()
fnc.savefig_dated(fig, f"pamspplot_validation_dualplot", type=plot_format, path=figure_path, bbox_inches="tight")